# 脱出構文と例外処理、大域脱出

# 脱出構文、大域脱出

* 処理を途中で抜ける方法
    * return文
    * ループを抜ける脱出構文
    * メソッドを抜ける大域脱出

## 脱出構文

* break
    * ループそのものを中断
* next
    * 中断して次に進む
    * 他の言語ではcontinueに相当
* redo
    現在の処理をやり直す

In [2]:
5.times do |i|
  next if i == 2
  puts i
end

### 2を表示した後2を無限ループで表示
# 5.times do |i|
#   redo if i == 2
#   puts i
# end

0
1
3
4


5

# 例外処理

## rise

* 例外を発生させる
* 第一引数
    * 例外クラス又はそのインスタンス
* 第二引数
    * メッセージ

In [4]:
raise ArgumentError, "引数が不正です(例外クラス)"

ArgumentError: 引数が不正です(例外クラス)

In [5]:
raise ArgumentError.new, "引数が不正です(例外クラスのインスタンス)"

ArgumentError: 引数が不正です(例外クラスのインスタンス)

In [10]:
### 例外クラスのコンクラスタでメッセージを指定
err = ArgumentError.new("引数が不正です(メッセージを含む例外オブジェクト)")
raise err

ArgumentError: 引数が不正です(メッセージを含む例外オブジェクト)

In [11]:
raise "例外クラスのインスタンスを省略した場合はRuntimeError"

RuntimeError: 例外クラスのインスタンスを省略した場合はRuntimeError

## 例外が発生しても処理を続行させる

* rescure節
    * 例外が発生したときに実行される処理を記述
* else節
    * 例外が発生しなかった時の処理を記述
* ensure節
    * 例外の発生如何に関わらず実行される

In [19]:
puts "rescue節"
begin
  1 / 0
  puts "例外の下の処理、実行されない"
rescue
  puts "例外が発生すると実行されるrescue節"
end

puts "else節"
begin
  1
  puts "例外は発生してない"
rescue
  puts "例外は発生してないのでrescue節は実行されない"
else
  puts "例外は発生していないので実行されるelse節"
end

puts "ensure節"
begin
  1 / 0
  puts "例外の下の処理、実行されない"
rescue
  puts "例外が発生すると実行されるrescue節"
else
  puts "例外が発生しているので実行されないelse節"
ensure
  puts "例外発生の如何に関わらず実行されるensure節"
end


rescue節
例外が発生すると実行されるrescue節
else節
例外は発生してない
例外は発生していないので実行されるelse節
ensure節
例外が発生すると実行されるrescue節
例外発生の如何に関わらず実行されるensure節


## beginを書かかずにrescue節を記述

In [22]:
1 / 0 rescue puts "begin無しrescue節"

begin無しrescue節


## メソッドの中でrescue節を実行

In [23]:
def foo
  1 / 0
rescue
  puts "メソッドの中でrescueを実行"
end
foo

メソッドの中でrescueを実行


# 例外クラスを指定した捕捉

* 例外クラスの捕捉
    * 例外クラスを指定せずにrescue節が補足する対象
        * StandardErrorとそのサブクラス
* 例外クラスの階層
* Exception
    * ScriptError
        * SyntaxError : 文法エラーがあった場合
    * SignalException : 捕捉していないシグナルを受けた場合
    * StandardError
        * ArgumentError : 引数の数が合わないときや値が正しくないとき
        * RuntimeError : 特定の例外クラスに該当しないエラー、もしくは例外クラスを省略してraiseを呼び出した場合
        * NameError : 未定義のローカル変数や定数を参照した場合
            * NoMethodError : 未定義のメソッドを呼び出した場合
        * ZeroDivisionError : 整数に対して整数の0で除算を行った場合

In [25]:
puts "例外オブジェクトの取得。backtraceで例外が発生した場所を参照"
begin
  1/0
rescue ZeroDivisionError => e
  puts e.backtrace
end

例外オブジェクトの取得
["(pry):158:in `/'", "(pry):158:in `<main>'", "/Users/ftakao2007/work/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-0.10.4/lib/pry/pry_instance.rb:355:in `eval'", "/Users/ftakao2007/work/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-0.10.4/lib/pry/pry_instance.rb:355:in `evaluate_ruby'", "/Users/ftakao2007/work/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-0.10.4/lib/pry/pry_instance.rb:323:in `handle_line'", "/Users/ftakao2007/work/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-0.10.4/lib/pry/pry_instance.rb:243:in `block (2 levels) in eval'", "/Users/ftakao2007/work/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-0.10.4/lib/pry/pry_instance.rb:242:in `catch'", "/Users/ftakao2007/work/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-0.10.4/lib/pry/pry_instance.rb:242:in `block in eval'", "/Users/ftakao2007/work/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-0.10.4/lib/pry/pry_instance.rb:241:in `catch'", "/Users/ftakao2007/work/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-0.10.4/lib/pry/pry_instance.rb:241:in 

## $! による最後の例外の取得と再発生

In [31]:
begin
  1/0
rescue ZeroDivisionError => e
  puts $!.class
  puts $!
  raise
end

ZeroDivisionError
divided by 0


ZeroDivisionError: divided by 0

## retry

* 例外が発生したらrescue節を全て実行した後、再度beginから処理を実効する

In [57]:

a = 0
b = 0

begin
  b = 6/a
rescue ZeroDivisionError
  puts "一回目は6/0で例外発生。rescure節でaに3が加えられてretryで再度beginから実行され6/3が計算される"
  puts b
  a += 3
  retry
ensure
  puts "2回目は例外発生せずensure節で計算結果の2が出力される"
  puts b
end

一回目は6/0で例外発生。rescure節でaに3が加えられてretryで再度beginから実行され6/3が計算される
0
2回目は例外発生せずensure節で計算結果の2が出力される
2


2

## rescue節の実行順

* rescue節は一つのbegin節の中にいくらでも書くことができる
* 最初にマッチしたものしか実行されない
    * <font color="red">マッチする範囲が広くなる順に指定する</font>
        * 後でより狭い例外クラスを指定しても役に立たない

In [58]:
begin
  1/0
rescue
  p 1
rescue ZeroDivisionError
  p 2
end

1


1

# 大域脱出

## catch/throw

* 正常時であっても処理を抜けたいとき
    * 階層の深いループの中で全ての処理が完了した場合など
* 例外との対応イメージ
    * raise <-> throw
    * begin節 <-> catch
* 処理の流れ
    * throwが実行される
    * 同名のラベル(ここでは:exit)が指定されているcatchまで呼び出してスタックをたどる
    * 見つかった場合はそのブロック内における後続の処理をスキップする
* ラベル
    * シンボルや文字列を指定できる
* ラベルが見つからない場合はNameErrorが発生

In [88]:
puts "catchにおいてthrowの設定されているfooメソッド(ラベルは:exit)以降は実行されない"
def foo
  puts "throwありメソッド"
  throw :exit
end

catch(:exit) {
  foo
  puts "catch内。実行されない"
}
puts "1"
puts "==="  

puts "barメソッドにthrowが設定されていないのでcatch内は全て実行される"
def bar
  puts "throw無しメソッド"
end

catch(:exit2) {
  bar
  puts "catch内。実行される"
}
puts "2"
puts "==="

puts "ラベルが一致しない場合はエラーになる"
def baz
  puts "throwとcatchのラベルが一致してない"
  throw :hoge
end

catch(:exit3) {
  baz
  puts "catch内。実行される"
}
puts "3"
  

catchにおいてthrowの設定されているfooメソッド(ラベルは:exit)以降は実行されない
throwありメソッド
1
===
barメソッドにthrowが設定されていないのでcatch内は全て実行される
throw無しメソッド
catch内。実行される
2
===
ラベルが一致しない場合はエラーになる
throwとcatchのラベルが一致してない


UncaughtThrowError: uncaught throw :hoge

## catchの戻り値をthrowに渡して実行

* throwに渡した値はcatchの戻り値とすることができる

In [95]:
puts "メソッドの中にcatchとthrowを書く"
def bar
  catch(:calc) do
    throw :calc, 100
  end
end
puts bar

puts "今までの例の書き方で実行"
def foo
  throw :exit, 200
end

catch(:exit) {
  puts foo
}

メソッドの中にcatchとthrowを書く
100
今までの例の書き方で実行


200